# Day 05: More Linear Regression
Review of last time:

* Linear regression is about creating a linear model to predict Y along variable(s) X
    * Involves getting estimates for coefficients Bn
    * Model is evaluated by residuals e
* Standard Error (SE) is the variance of Bn
* 95% Confidence Interval = Bn ± 2(SE)
* Hypothesis Testing:
    * Prove the null hypothesis (Y can not be explained by X) is wrong
    * Use result of P-value to reject or not, based on value of 0.05
* Smaller p-value means stronger relationship, larger p-value means that random chance is more likely
* RSE measure lack of fit (smaller is better)
* R Squared measures how much of the data can be explained by regression
    * 1 = all data can be explained
    * 0 = no data can be explained
* R Squared = RSS/TSS
    * RSS (Residual Sum of Squares) = SUM of Residuals (difference between actual and predicted y)
    * TSS (Total Sum of Squares) = SUM of Variation (difference between actual and mean of y)
    * We want to minimize RSS

## Choosing subset
It is possible that not all variables are good for predicting. Maybe choosing particular ones (a subset) is better than including all?

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

In [2]:
insurance = pd.read_csv('insurance.csv')

# All numerical variables
insurance_all = smf.ols('charges ~ age + bmi + children', insurance).fit()
insurance_all.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     60.69
Date:                Sun, 09 Nov 2025   Prob (F-statistic):           8.80e-37
Time:                        22:22:50   Log-Likelihood:                -14392.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1334   BIC:                         2.881e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -6916.2433   1757.480     -3.935      0.000   -1.04e+04   -3468.518
age          239.9945     22.289     10.767      0.000     196.269     283.720
bmi          332.0834     51.310      6.472      0.000     231.425     432.741
children     542.8647    258.241      2.102      0.036      36.261    1049.468
==============================================================================
Omnibus:                      325.395   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              603.372
Skew:                           1.520   Prob(JB):                    9.54e-132
Kurtosis:                       4.255   Cond. No.                         290.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

There are three numerical variables. And so there are 6 other subsets:
* age only
* bmi only
* children only
* age and bmi
* age and children
* bmi and children

We can compare model performance by way of RSS.

In [3]:
def statsmodelRSS(est):
    # Returns the RSS for the statsmodel ols class
    return np.sum(est.resid**2)

In [4]:
ins_age = smf.ols('charges ~ age', insurance).fit()
print(statsmodelRSS(ins_age))
ins_bmi = smf.ols('charges ~ bmi', insurance).fit()
print(statsmodelRSS(ins_bmi))
ins_child = smf.ols('charges ~ children', insurance).fit()
print(statsmodelRSS(ins_child))
ins_age_bmi = smf.ols('charges ~ age + bmi', insurance).fit()
print(statsmodelRSS(ins_age_bmi))
ins_age_child = smf.ols('charges ~ age + children', insurance).fit()
print(statsmodelRSS(ins_age_child))
ins_bmi_child = smf.ols('charges ~ bmi + children', insurance).fit()
print(statsmodelRSS(ins_bmi_child))

178544029385.2155
188360830331.80313
195167621650.2592
173097580364.0642
177943340984.38782
187520317725.7104


The subset with the lowest RSS is `age` and `bmi`, with a RSS of 173097580364.0642.\
Let's compare this subset's perfomance with the earlier example of using all numerical variables.

In [5]:
print(statsmodelRSS(insurance_all))

172526061322.4756


Including the variables `children` will lower the RSS, so in this case, having all numerical variables will help.

### But what about including the other variables? A patient's sex, smoker status, and residential region are also avaible to analyze... ###
These variables are called *qualitative*, or *categorical*. Qualitative variables are non-numerical, such as unordered sets or booleans. This is in contrast to the quantitative (numerical) variables we were working with earlier.

Qualitative variables cannot be directly worked with in regression. They have to be converted into a numerical format. This can be done by *dummy variables* - dividing each categorical answer by introducing a 1 for presence.

A quick demonstration on the `sex` variable:

In [6]:
insurance[['sex']]

,sex
0,female
1,male
2,male
3,male
4,male
...,...
1333,male
1334,female
1335,female
1336,female


In [7]:
# True and False
pd.get_dummies(insurance['sex'])

,female,male
0,True,False
1,False,True
2,False,True
3,False,True
4,False,True
...,...,...
1333,False,True
1334,True,False
1335,True,False
1336,True,False


In [8]:
# Convert true and falso to numerical
pd.get_dummies(insurance['sex'], dtype=float)

,female,male
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
1333,0.0,1.0
1334,1.0,0.0
1335,1.0,0.0
1336,1.0,0.0


### Categorical variables in action
Let's repeat the linear regression analysis by introducing our categorical variables into the model. In our `insurance` data, this will be the patient's sex (Male or Female), whether the patient smokes or not (Yes, No), and regeion of the country they live (Northeast, Northwest, Southeast, Southwest).\
Ok, time to convert dummies into the full data.

In [9]:
X = insurance.drop(['charges'], axis=1) # drop the charges column
X = pd.get_dummies(X, dtype=float) # Get dummies
X

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,18,33.770,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,28,33.000,3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33,22.705,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,32,28.880,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1334,18,31.920,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1335,18,36.850,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1336,21,25.800,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


The response `charges` will be *y*.

In [10]:
y = insurance['charges']

Using **Sklearn**

In [11]:
# Fit the model
full = LinearRegression().fit(X, y)

In [12]:
for pair in range(len(X.columns)):
    print(X.columns[pair], ':', full.coef_[pair])
print('Intercept', full.intercept_)

age : 256.85635253734847
bmi : 339.1934536108377
children : 475.50054514913285
sex_female : 65.6571796975502
sex_male : -65.6571796975514
smoker_no : -11924.267270956405
smoker_yes : 11924.267270956401
region_northeast : 587.0092350283238
region_northwest : 234.04533560368452
region_southeast : -448.0128143595046
region_southwest : -373.04175627250305
Intercept -666.9377199366318


Using **Statsmodels**

In [13]:
full_mod2 = smf.ols('charges ~ age + sex + bmi + children + smoker + region', insurance).fit()

In [14]:
full_mod2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Sun, 09 Nov 2025   Prob (F-statistic):               0.00
Time:                        22:22:50   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -1.194e+04    987.819    -12.086      0.000   -1.39e+04      -1e+04
sex[T.male]          -131.3144    332.945     -0.394      0.693    -784.470     521.842
smoker[T.yes]        2.385e+04    413.153     57.723      0.000     2.3e+04    2.47e+04
region[T.northwest]  -352.9639    476.276     -0.741      0.459   -1287.298     581.370
region[T.southeast] -1035.0220    478.692     -2.162      0.031   -1974.097     -95.947
region[T.southwest]  -960.0510    477.933     -2.009      0.045   -1897.636     -22.466
age                   256.8564     11.899     21.587      0.000     233.514     280.199
bmi                   339.1935     28.599     11.860      0.000     283.088     395.298
children              475.5005    137.804      3.451      0.001     205.163     745.838
==============================================================================
Omnibus:                      300.366   Durbin-Watson:                   2.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              718.887
Skew:                           1.211   Prob(JB):                    7.86e-157
Kurtosis:                       5.651   Cond. No.                         311.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Statsmodels will automatically convert the categorical features for you. Note that instead of creating a variable for each level in a category, they will instead use the first alphabetical category as a base/intercept, and instead create new levels for the remaining features.

For example, there only exists a seperate sex variable for *male*. They have now used the *female* variable as a base, and thus defaults to a 0 when using the coefficient.

Looking at the p-values, it appears that the sex and northwest region variables are above the 0.05 threshold and do not show significance. Perhaps they do not have an impact on predicting the charge?